# Liga 1 Perú 2022: Base de datos

El siguiente script y base de datos responde a la falta de una base de datos totalizada que reporte información diversificada sobre los partidos de la primera división del futbol peruano y aportar con ideas para solucionar problemas que se reportan al elaborar 'scrapers'. Se espera que la base de datos resultante motive la exploración, análisis y desarrollo en temas relacionado con el sport analytics en el futbol peruano. Además, satisfacer la falta de una base de datos con variedad de datos y gratuita. La base de datos no puede ser usada con fines comerciales.

Si tiene alguna sugerencia o comentario, contactar mediante:
- @: gianfranco.soria@pucp.edu.pe
- GitHub: GianfrancoSoria
- Twitter: @SoriaGianfranco

*_La Liga 1 edición 2022 aún no termina, los siguientes datos están actualizados al 06/08/22._

## Librerías necesarias

En esta sección se importan todas las librerías que serán necesarias para el desarrollo del código más adelante.

In [1]:
#!pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
import numpy as np
import threading

## Evaluación de la página

La página seleccionada para ser la fuente de datos es [Flashscore](https://www.flashscore.pe/futbol/peru/liga-1-2021/resultados/). La elección se debió a que la página ofrece los datos de forma gratuita, no tiene bloqueos o limitaciones y presenta una estructura idónea para la extracción de datos. 

Tras la evaluación de la página se encuentra que los resultados se muestran en dos partes. En la primera se muestran los resultados de diferentes partidos, pero solo algunos datos (fecha, número de goles y tarjetas rojas). En la segunda se muestra el detallado de algún partido seleccionado, los datos que se muestran son variados (goles, tarjetas amarillas, offside, etc.). Sin embargo, se reconocen algunos retos para ambas etapas. En la primera etapa se reconoce que no se muestran todos los partidos, solo algunos. Para acceder a todos se debe cargar los datos y para ello se requiere desplazar la página hasta el final y hacer clic en la opción "Mostrar más partidos". En la segunda etapa se observa que los vínculos (que se encuentran en la primera etapa) están ocultos, pero se identifica una estructura común en la dirección URL con "códigos identificadores" para cada partido.

Solucionar los retos de la primera parte se emplean los comandos "driver.execute_script" (rodar el scroll de la página) y "driver.find_element" (ubicar y hacer clic en "Mostrar más partidos"). En cuanto a los retos de la segunda parte resultan más difícil de solucionar, mas no imposibles. Para extraer los "códigos identificadores" emplearemos una estrategia similar a si usáramos la librería beautifulsoup. Delimitamos que estos códigos se encuentran entre 'div id="g_1_' y 'title="¡Haga click para detalles del partido!"'. Empleando esta restricción almacenamos los resultados en una lista con nombre "result_buscar".

In [5]:
#---------------------------------------------------------------------------------------------------------------------------------------------
# Apertura de página
#---------------------------------------------------------------------------------------------------------------------------------------------
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.flashscore.pe/futbol/peru/liga-1-2022/resultados/") 
driver.find_element(By.XPATH, "//*[@id='onetrust-accept-btn-handler']").click()
driver.maximize_window()
time.sleep(2)
#---------------------------------------------------------------------------------------------------------------------------------------------
# Retos primera etapa
#---------------------------------------------------------------------------------------------------------------------------------------------
driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
driver.find_element(By.XPATH, "//*[@id='live-table']/div[1]/div/div/a").click()
time.sleep(2)
driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
driver.find_element(By.XPATH, "//*[@id='live-table']/div[1]/div/div/a").click()
time.sleep(2)
driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
#---------------------------------------------------------------------------------------------------------------------------------------------
# Retos segunda etapa
#---------------------------------------------------------------------------------------------------------------------------------------------
content = driver.find_element(By.XPATH, "//*[@id='live-table']/div[1]/div/div")
soup=content.get_attribute('innerHTML')
buscar1='<div id="g_1_'
result1 = [_.start() for _ in re.finditer(buscar1, soup)]
buscar2='title="¡Haga click para detalles del partido!"'
result2 = [_.start() for _ in re.finditer(buscar2, soup)]
result_buscar=[0]*len(result1)
for i in range(len(result1)):
    result_buscar[i]=str(soup[result1[i]+13:result2[i]-2])
driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
C:\Users\PC-1\AppData\Local\Temp\ipykernel_24140\3501453514.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Primeros resultados: Árbitros

Empleando los "códigos identificadores" extraídos anteriormente podemos visualizar las páginas personalizadas para cada partido. Estos resultados muestran 4 opciones (Partido, Cuotas, H2H y Cuadro). Los datos de interés se encuentran en la opción "Partido" y dentro de las sub opciones "Resumen" (nombre del árbitro) y "Estadísticas" (fecha, número de goles y tarjetas rojas). Como primer ejercicio de exploración nos centraremos en la extracción del nombre del árbitro. Existe un caso especial, el identificador 148 no tiene el mismo XPATH. Por lo que se crea un condicional 'if'. Por otro lado, identificamos que el tiempo mínimo para para cada código identificador es de 9.5 segundos (contando solo los sleep) y ,por ello, el tiempo total mínimo sería 38 minutos. Este tiempo podría considerarse como extenso y representa un reto computacional importante. Para sobrepasar este reto usaremos la librería 'threading' que nos permite usar más procesadores o hilos y estará limitado por el número que tenga nuestro procesador. Emplear más hilos o procesadores nos permite ejecutar múltiples tareas al mismo tiempo. Aplicado a este caso, implica que podremos scrapear varios códigos identificadores al mismo tiempo. El número de hilos usados en el presente proceso fue de 15 y el tiempo total fue 5.0 minutos. Este tiempo se vera afectado por el número de hilos y la conexión a internet.

In [33]:
arbitro=[0]*len(result_buscar)
def referee(min_i,max_i):
    for i in range(min_i,max_i):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get('https://www.flashscore.pe/partido/'+f'{result_buscar[i]}'+'/#/resumen-del-partido/resumen-del-partido')
        time.sleep(6)
        driver.find_element(By.XPATH, "//*[@id='onetrust-accept-btn-handler']").click()
        driver.maximize_window()
        time.sleep(0.5)
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        if i==148:
            arbitro[i]=driver.find_element(By.XPATH,"//*[@id='detail']/div[10]/div[2]/div/span[2]").text #Revisar aquí por actualizaciones    
        elif i==156:
            arbitro[i]=driver.find_element(By.XPATH,"//*[@id='detail']/div[11]/div[2]/div[1]/span[2]").text #Revisar aquí por actualizaciones
        else:
            arbitro[i]=driver.find_element(By.XPATH,"//*[@id='detail']/div[10]/div[2]/div[1]/span[2]").text #Revisar aquí por actualizaciones
        time.sleep(3)
        driver.close()
        print('Termino '+str(i))
        print(arbitro)
#---------------------------------------------------------------------------------------------------------------------------------------------
# Número total de threads usados
n_threads=15
#---------------------------------------------------------------------------------------------------------------------------------------------
p=len(result_buscar)//n_threads
inicios=list()
finales=list()
inicio=0
fin=p
for i in range(n_threads):
    inicios.append(inicio)
    finales.append(fin)
    inicio+=p
    fin+=p
time_inicio = time.time()
threads=list()
for j in range(len(inicios)):
    t=threading.Thread(target=referee, args=(inicios[j],finales[j],))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
print(' ')
print('--------------------------------------------------------------------------------------------------------------------------------------')
print('El tiempo total fue de '+str(round(time.time()-time_inicio,2))+' segundos.')
print('--------------------------------------------------------------------------------------------------------------------------------------')

[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-c

...

Termino 82
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)'

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Termino 180
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Termino 97
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)'

Algunos "codigos identificadores" extraidos quedaron fuera del rango de los threads. El tiempo que le tomo fue de 2.66 minutos para 8 casos, empleando una regla de tres simples determinamos que el tiempo total, sin threads, hubiera sido de 72.82 minutos. 

In [34]:
#Caso que quedo fuera de los rangos
time_inicio = time.time()
referee(210,219)
print(' ')
print('--------------------------------------------------------------------------------------------------------------------------------------')
print('El tiempo total fue de '+str(round(time.time()-time_inicio,2))+' segundos.')
print('--------------------------------------------------------------------------------------------------------------------------------------')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
C:\Users\PC-1\AppData\Local\Temp\ipykernel_24140\2968724897.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Termino 210
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Termino 211
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


...

Termino 216
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Termino 217
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Termino 218
['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)

Explorando los resultados encontramos que para algunos casos se extrae el nombre del estadio y no del árbitro. La razón de este hecho es que para dichos partidos no se registró el nombre del árbitro, por lo que no hay acción correctiva alguna. Lo que sí se puede corregir es cambiar dichos nombres por valores faltantes, representados por "-". Identificamos estos nombres en la lista "agregar_NaN".

In [35]:
agregar_NaN=['Estadio Monumental (Lima)','Estadio Huancayo (Huancayo)','Estadio Inca Garcilaso de la Vega (Cusco)',
            'Estadio Miguel Grau (Piura)','Estadio Miguel Grau (Callao)','Estadio San Martin de Porres (Lima)',
            'stadio Iván Elías Moreno (Lima)','Estadio Alejandro Villanueva (Lima)','Estadio Iván Elías Moreno (Lima)']
for j in agregar_NaN:
    for i in range(len(arbitro)):
        if arbitro[i]==j:
            arbitro[i]='-'
print(arbitro)

['Perez R. B. (Per)', 'Quiroz J. C. (Per)', 'Villegas H. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Menendez A. (Per)', 'Espinoza J. (Per)', 'Vasquez B. (Per)', 'Huamani J. (Per)', 'Lopez Ramos P. (Esp)', 'Garay L. (Per)', 'Ureta D. (Per)', 'Zamora J. (Per)', 'Alarcon J. (Per)', 'Cartagena J. (Per)', 'Villegas H. (Per)', 'Cori V. (Per)', 'Villanueva A. (Per)', 'Legario F. (Per)', 'Blas W. (Per)', 'Seminario L. (Per)', 'Perez R. B. (Per)', 'Ureta D. (Per)', 'Lozano S. (Per)', 'Ortega K. (Per)', 'Garay L. (Per)', 'Lopez Ramos P. (Esp)', 'Segura R. (Per)', 'Espinoza M. (Per)', 'Ordonez E. (Per)', 'Quiroz J. C. (Per)', 'Alarcon J. (Per)', 'Perez R. B. (Per)', 'Espinoza M. (Per)', 'Ortega K. (Per)', 'Ordonez E. (Per)', 'Espinoza J. (Per)', 'Huamani J. (Per)', 'Blas A. (Per)', 'Lopez P. (Per)', 'Seminario L. (Per)', 'Barbagelatta G. (Per)', 'Garay Evia L. (Per)', 'Lozano S. (Per)', 'Huaman D. (Per)', 'Vasquez B. (Per)', 'Legario F. (Per)', 'Villegas H. (Per)', 'Garay L. (Per)', 'Ureta D.

Explorando los resultados se determinó que el número de NaN que se han colocado es 12 y representa el 7.76%. Asimismo, en la siguiente tabla se explora el número de partidos por arbitro que se registra.

In [39]:
values, counts = np.unique(arbitro, return_counts=True)
data = {'Arbitro': list(values), 'Partidos': list(counts)}
tabla_arbitro=pd.DataFrame(data).sort_values('Partidos',ascending=False).reset_index(drop=True)
print('La cantidad de NaN es '+str(tabla_arbitro.Partidos[6])+' ('+ str(round(tabla_arbitro.Partidos[0]/tabla_arbitro.Partidos.sum()*100,2))+' %).')
tabla_arbitro.drop(tabla_arbitro.index[[7]]).reset_index(drop=True)

La cantidad de NaN es 12 (7.76 %).


,Arbitro,Partidos
0,Alarcon J. (Per),17
1,Perez R. B. (Per),16
2,Ordonez E. (Per),13
3,Espinoza M. (Per),13
4,Legario F. (Per),12
5,Cartagena J. (Per),12
6,Espinoza J. (Per),12
7,Villegas H. (Per),10
8,Ortega K. (Per),10
9,Seminario L. (Per),9


## Extracción de multiples datos y creación de base

En esta sección nos centraremos en extraer datos relacionados con el partido y equipos. Los datos de interés se encuentran en una sub pestaña de nombre "Estadísticas". Explorando las opciones para acceder a esta pestaña se optó por proceder mediante la dirección "url". Para ello necesitamos añadir a la dirección anteriormente usada el siguiente texto: '/#/resumen-del-partido/estadisticas-del-partido/0'. Tras explorar las diversas opciones para extraer los datos y las similitudes con las secciones anteriores se diseñó una estrategia muy similar. Se crearon tres bases previas para almacenar los datos: dt_local, almacenara los datos del equipo local; dt_visita, almacenara los datos del equipo de visita y key_list, almacena el nombre de las variables de interés. El primer reto que se encontró está relacionado con las variables de interés puesto que dichas variables no se reportan o mantienen el mismo orden. Para ello se optó por extraer toda la sección y extraer los datos de manera similar que para los árbitros. Para fijar los limites transformamos la lista de almacenada como texto a lista, separado por '\n', y la lista 'key_list'. En conjunto nos permiten separar las variables de interés y fijar un orden determinado. Por otro lado, se mantuvo la estrategia del uso de "threads" por razones antes expuestas en la sección anterior. El número de hilos usados en el presente proceso fue de 15 y el tiempo total fue 6.13 minutos.

In [43]:
dt_local=pd.DataFrame(columns=['Fecha','Local','GL','Posesión de balón','Remates','Remates a puerta','Remates fuera','Remates rechazados','Córneres',
         'Fueras de juego','Paradas','Faltas','Tarjetas rojas','Tarjetas amarillas','Pases totales','Tackles',
          'Ataques','Ataques peligrosos'], index=range(len(result_buscar)))
dt_visita=pd.DataFrame(columns=['Visita','GV','Posesión de balón','Remates','Remates a puerta','Remates fuera','Remates rechazados','Córneres',
         'Fueras de juego','Paradas','Faltas','Tarjetas rojas','Tarjetas amarillas','Pases totales','Tackles',
          'Ataques','Ataques peligrosos'], index=range(len(result_buscar)))
key_list=['Posesión de balón','Remates','Remates a puerta','Remates fuera','Remates rechazados','Córneres',
         'Fueras de juego','Paradas','Faltas','Tarjetas rojas','Tarjetas amarillas','Pases totales','Tackles',
          'Ataques','Ataques peligrosos']

In [45]:
def team(min_i,max_i):
    for i in range(min_i,max_i):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get('https://www.flashscore.pe/partido/'+f'{result_buscar[i]}'+'/#/resumen-del-partido/estadisticas-del-partido/0')
        time.sleep(8)
        driver.find_element(By.XPATH, "//*[@id='onetrust-accept-btn-handler']").click()
        driver.maximize_window()
        time.sleep(0.5)
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        soup=driver.find_element(By.XPATH,'//*[@id="detail"]/div[8]').text.split('\n')
        dt_local.loc[i,'Fecha']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[1]/div").text
        dt_local.loc[i,'Local']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[2]/div[3]/div[2]/a").text
        dt_visita.loc[i,'Visita']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[4]/div[3]/div[1]/a").text
        dt_local.loc[i,'GL']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[3]/div/div[1]/span[1]").text
        dt_visita.loc[i,'GV']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[3]/div/div[1]/span[3]").text
        key_list_ind=list()
        key_list_name=list()
        for j in range(len(key_list)):
            if key_list[j] in soup:
                key_list_ind.append(soup.index(key_list[j]))
                key_list_name.append(key_list[j])
        for k in range(len(key_list_ind)):
            dt_local.loc[i,key_list_name[k]]=soup[key_list_ind[k]-1]
            dt_visita.loc[i,key_list_name[k]]=soup[key_list_ind[k]+1]
        time.sleep(5)
        driver.close()
#---------------------------------------------------------------------------------------------------------------------------------------------
# Número total de threads usados
n_threads=15
#---------------------------------------------------------------------------------------------------------------------------------------------p=len(result_buscar)//n_threads
inicios=list()
finales=list()
inicio=0
fin=p
for i in range(n_threads):
    inicios.append(inicio)
    finales.append(fin)
    inicio+=p
    fin+=p
time_inicio = time.time()
threads=list()
for j in range(len(inicios)):
    t=threading.Thread(target=team, args=(inicios[j],finales[j],))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
print(' ')
print('--------------------------------------------------------------------------------------------------------------------------------------')
print('El tiempo total fue de '+str(round(time.time()-time_inicio,2))+' segundos.')
print('--------------------------------------------------------------------------------------------------------------------------------------')

[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Get LATEST chromedriver version for 104.0.511

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


...

[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome


[WDM] - ====== WebDriver manager ======
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


 
--------------------------------------------------------------------------------------------------------------------------------------
El tiempo total fue de 367.99 segundos.
--------------------------------------------------------------------------------------------------------------------------------------


Algunos "codigos identificadores" extraidos quedaron fuera del rango de los threads. El tiempo que le tomo fue de 3.28 minutos para 8 casos, empleando una regla de tres simples determinamos que el tiempo total, sin threads, hubiera sido de 89.8 minutos. 

In [48]:
#Caso que quedo fuera de los rangos
time_inicio = time.time()
team(210,219)
print(' ')
print('--------------------------------------------------------------------------------------------------------------------------------------')
print('El tiempo total fue de '+str(round(time.time()-time_inicio,2))+' segundos.')
print('--------------------------------------------------------------------------------------------------------------------------------------')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
C:\Users\PC-1\AppData\Local\Temp\ipykernel_24140\2540876866.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


 
--------------------------------------------------------------------------------------------------------------------------------------
El tiempo total fue de 196.64 segundos.
--------------------------------------------------------------------------------------------------------------------------------------


Como se había mencionado, se crearon dos data frame para almacenar los resultados ('dt_visita' y 'dt_local'). Ambos data frame se unirán en uno solo de nombre 'dt', con la finalidad de tener una base totalizada.

In [49]:
dt_visita.columns=['Visita','GV','PV','RemV','RpV','RfV','RbV','CV',
                    'OffV','ArqV','FV','RV','AV','TpV','BkV','AtaqV',
                   'DAtaqV']
dt_local.columns=['Fecha','Local','GL','PL','RemL','RpL','RfL','RbL','CL',
                   'OffL','ArqL','FL','RL','AL','TpL','BkL','AtaqL',
                   'DAtaqL']
#---------------------------------------------------------------------------------------------------------------------------------------------
# Colapsar bases de datos (local y visitante) en una sola
dt=pd.merge(dt_local, dt_visita, left_index=True, right_index=True).reindex(columns=['Fecha','Local','Visita','GL','GV','PL','PV','RemL','RemV',
                                                                                     'RpL','RpV','RfL','RfV','RbL','RbV','CL','CV','OffL','OffV',
                                                                                     'ArqL','ArqV','FL','FV','RL','RV','AL','AV','TpL','TpV',
                                                                                     'BkL','BkV','AtaqL','AtaqV','DAtaqL','DAtaqV'])
#---------------------------------------------------------------------------------------------------------------------------------------------

Explorando el data frame 'dt' se encontró que las variables relacionadas con las tarjetas del partido presentaban una múltiple cantidad de 'NaN'. Esto se explica debido a que no en todos los partidos se reportan tarjetas amarillas o rojas (especialmente). Estas 'NaN' fueron reemplazadas por el valor de 0, puesto que mantiene la premisa anterior. Por otro lado, las otras variables también reportaron 'NaN', sin embargo, no se puede reemplazar por el valor de 0. Puesto que no necesariamente siguen la misma lógica que las tarjetas del partido. Por lo que las demás 'NaN' fueron reemplazadas por '-' para no alterar su significado. Por último, se agregó la información de árbitros a la base totalizada.

In [50]:
# Últimos arreglos
dt['AL']=dt['AL'].fillna(0)
dt['AV']=dt['AV'].fillna(0)
dt['RL']=dt['RL'].fillna(0)
dt['RV']=dt['RV'].fillna(0)
dt=dt.fillna('-')
dt.insert(1,'Arbitro',arbitro)

## Exportar base de datos

In [51]:
dt.to_excel('Liga1 (Perú) 2022.xlsx', index=False, sheet_name='2022')
dt.to_csv('Liga1 (Perú) 2022.csv', index=False, sep=';')

## Nota final

Durante el desarrollo del presente script se realizaron hasta dos actualizaciones en el código relacionados con los XPATH. Por lo que es importante realizar un seguimiento al código para que no pierda validez. Asimismo, se ha encontrado que la estructura html no se mantiene para otros años. Por lo que si se quisiera obtener información de otro año se recomienda tomar como base el presente código y adaptar la nueva estructura html (XPATH por lo general).